### Title: 
# Exploratory Data Analysis

### Description:
We are going to perform an EDA of our dataset.

### Authors:
#### Hugo Cesar Octavio del Sueldo
#### Jose Lopez Galdon

### Date:
23/10/2020

### Version:
1.0

***

### Libraries

We are going to start this EDA loading the respective libraries

In [3]:
    # Numpy & Pandas to work with the DF
import numpy as np
import pandas as pd

    # Seaborn / matplotlib for graphs
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

    # Automatic reports
from pandas_profiling import ProfileReport

    # Visualize DF
from IPython.display import display, HTML

***

## Load data

In [4]:
    # To automate the work as much as possible, we will parameterize the codes, so in this case, we will create an objetct with
    # the path root
name = 'data_train_merge'

data = pd.read_csv(f'../data/02_intermediate/{name}.csv',  # Path root: here we include an f-string with the variable name to parametrize
                   low_memory = False)                     # To avoid warnings we use set low_memory = False

***

## Variable predict

Taking into account the goals of the practice, we only want to know those persons that `Fully paid` their loans or those who does not, so we will tansform de dependent variable into binary problem, to ones or ceros depending on that condition. 

In [10]:
    # We will only select those observations with "Fully Paid" & "Charged Off"
data_binary = data[(data['loan_status'] == "Fully Paid") | (data['loan_status'] == "Charged Off")]

    # Now, we will transform into 0 & 1
dummy_dict = {"Fully Paid":0, "Charged Off":1}

    # Finally, we use the dictiony in the dataset
data = data_binary.replace({"loan_status": dummy_dict})

In [13]:
data

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
8,112038251,NaN,11575,11575,11575.0,36 months,7.35%,359.26,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
10,112149045,NaN,7200,7200,7200.0,36 months,24.85%,285.70,E,E3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
24,112052261,NaN,7500,7500,7500.0,36 months,7.35%,232.79,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
42,111999259,NaN,10000,10000,10000.0,60 months,16.02%,243.29,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
91,111808508,NaN,14000,14000,14000.0,36 months,16.02%,492.34,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884869,36221446,NaN,11575,11575,11575.0,36 months,15.59%,404.61,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
884870,36330911,NaN,12000,12000,12000.0,36 months,11.99%,398.52,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
884873,36271333,NaN,13000,13000,13000.0,60 months,15.99%,316.07,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
884874,36490806,NaN,12000,12000,12000.0,60 months,19.99%,317.86,E,E3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


### View data

Firstly, we are goint to take a look to our dataframe:

In [14]:
    # First 10 rows using html display in order to view all the columns
data = data
display(HTML(data.head(10).to_html()))

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
8,112038251,NaN,11575,11575,11575.0,36 months,7.35%,359.26,A,A4,Solutions Architect,6 years,OWN,153000.0,Not Verified,Jun-2017,0,n,https://lendingclub.com/browse/loanDetail.action?loan_id=112038251,NaN,credit_card,Credit card refinancing,923xx,CA,16.99,0,Jul-1994,720,724,0,24.0,84.0,20,1,8550,22.7%,46,w,0.0,0.0,11707.816236,11707.82,11575.0,132.82,0.0,0.0,0.0,Sep-2017,11360.38,NaN,Sep-2017,724,720,0,24.0,1,Individual,NaN,NaN,NaN,0,0,442731,1.0,1.0,0.0,0.0,27.0,92315.0,63.0,2.0,8.0,1581.0,36.0,37600,1.0,6.0,2.0,8,23302.0,24269.0,13.6,0,0,130.0,275,3,3,2,3.0,NaN,7.0,NaN,0,6,9,9,16,12,16,32.0,9,20,0.0,0,0,2,95.7,11.1,1,0,528172,100865,28100,120572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
10,112149045,NaN,7200,7200,7200.0,36 months,24.85%,285.70,E,E3,Pse,2 years,RENT,50000.0,Source Verified,Jun-2017,0,n,https://lendingclub.com/browse/loanDetail.action?loan_id=112149045,NaN,debt_consolidation,Debt consolidation,985xx,WA,6.07,0,Jan-2000,685,689,0,72.0,NaN,4,0,3560,98.9%,5,w,0.0,0.0,7206.461000,7206.46,7200.0,6.46,0.0,0.0,0.0,Jul-2017,7231.31,NaN,Sep-2017,669,665,0,72.0,1,Individual,NaN,NaN,NaN,0,0,5588,0.0,1.0,0.0,1.0,21.0,2028.0,51.0,0.0,1.0,2779.0,74.0,3600,0.0,0.0,0.0,2,1863.0,40.0,98.9,0,0,21.0,172,14,14,0,14.0,NaN,14.0,NaN,1,2,2,3,3,1,3,3.0,2,4,0.0,0,0,0,80.0,100.0,0,0,7600,5588,3600,4000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
24,112052261,NaN,7500,7500,

In [15]:
    # Last 10 rows using html display in order to view all the columns
data = data
display(HTML(data.tail(10).to_html()))

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
884863,36241316,NaN,6225,6225,6225.0,36 months,16.49%,220.37,D,D3,Painter,2 years,RENT,27000.0,Source Verified,Jan-2015,0,n,https://lendingclub.com/browse/loanDetail.action?loan_id=36241316,NaN,debt_consolidation,Debt consolidation,330xx,FL,18.58,0,Feb-2011,680,684,1,NaN,NaN,3,0,1756,97.6%,4,f,0.0,0.0,7050.460000,7050.46,6225.00,825.46,0.0,0.00,0.0000,Dec-2015,4858.17,NaN,Dec-2015,684,680,0,NaN,1,Individual,NaN,NaN,NaN,0,0,8357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1800,NaN,NaN,NaN,1,2786.0,44.0,97.6,0,0,32.0,45,28,18,0,28.0,NaN,5.0,NaN,0,2,2,2,2,2,2,2.0,2,3,0.0,0,0,0,100.0,100.0,0,0,14307,8357,1800,12507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
884864,36250208,NaN,13150,13150,13150.0,60 months,14.99%,312.77,C,C5,Team Leader,8 years,MORTGAGE,30000.0,Verified,Jan-2015,0,n,https://lendingclub.com/browse/loanDetail.action?loan_id=36250208,NaN,major_purchase,Major purchase,286xx,NC,1.00,0,May-1994,825,829,0,NaN,NaN,3,0,672,3.2%,14,w,0.0,0.0,15669.110000,15669.11,13150.00,2519.11,0.0,0.00,0.0000,Jun-2016,10686.69,NaN,Mar-2015,819,815,0,NaN,1,Individual,NaN,NaN,NaN,0,0,158977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20700,NaN,NaN,NaN,0,52992.0,20028.0,3.2,0,0,NaN,246,70,47,3,70.0,NaN,NaN,NaN,0,1,1,2,7,0,2,11.0,1,3,0.0,0,0,0,100.0,0.0,0,0,190700,672,20700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
884865,36231724,NaN,12000,12000,12000.0,36 months,9.49%,

In [5]:
    # Data dimension
data.shape

(884876, 151)

### Data wrangling

Once we have viewed our dataset, we continue exploring the column types and nulls...

In [6]:
data.info(verbose = True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884876 entries, 0 to 884875
Data columns (total 151 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   id                                          884876 non-null  int64  
 1   member_id                                   0 non-null       float64
 2   loan_amnt                                   884876 non-null  int64  
 3   funded_amnt                                 884876 non-null  int64  
 4   funded_amnt_inv                             884876 non-null  float64
 5   term                                        884876 non-null  object 
 6   int_rate                                    884876 non-null  object 
 7   installment                                 884876 non-null  float64
 8   grade                                       884876 non-null  object 
 9   sub_grade                                   884876 non-null  object 


As we can see above the only one column that has only nulls is `member_id`, therefore we will proceed to drop it. Also we observed that we have multiple columns with NaN, such as `desc`, `open_il_12m`, `open_rv_12m`...

After inspection we conclude that the following columns are not important for the analysis. We will describe why below.
- `url`: This variable does not provide relevant information because the url does not include information of the borrower.
- `desc`: As this variable is plenty of null and is very similar to `purpose` we will drop it.
- `title`: Has the same information as `purpose`.
- `open_acc_6m`, `open_act_il`...: According to some authors from *Towards Data Science* it is a good practice to drop those columns that has more than the 50% with NaN.

In [15]:
    # Drop columns
data = data.drop(columns = ['member_id', 'url', 'desc', 'title', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 
                            'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',  'open_rv_24m', 'max_bal_bc', 
                            'all_util', 'num_tl_120dpd_2m', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 
                            'last_pymnt_d', 'next_pymnt_d'])

In [16]:
    # Drop rows
data = data.dropna(subset = ['emp_title', 'emp_length', 'last_credit_pull_d', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 
                             'mo_sin_old_il_acct', 'mths_since_recent_bc', 'num_rev_accts', 'dti'])

In [17]:
    # Fill nulls with 0
        # First  of all we create an object call columns that include all the variables we will fill 0 value
columns = ['mths_since_last_delinq','mths_since_last_record', 'mths_since_last_major_derog', 'inq_fi', 'total_cu_tl', 
           'inq_last_12m', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 
           'percent_bc_gt_75', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 
           'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 
           'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 
           'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog', 'hardship_type', 
           'hardship_reason', 'hardship_status', 'deferral_term',  'hardship_amount', 'hardship_start_date', 
           'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 
           'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',  
           'hardship_last_payment_amount', 'debt_settlement_flag_date', 'settlement_status',  'settlement_date',  
           'settlement_amount', 'settlement_percentage', 'settlement_term']
        
data[columns] = data[columns].fillna(value = 0)

In [18]:
data.shape

(801087, 130)

Now our dataset has 130 columns instead of 151.

Next, we will take a look at duplicates, so if we find something we will delete it.

In [8]:
data.duplicated().sum()

0

As we can see, we have any duplicated data.

In [25]:
data = data
display(HTML(data.tail(10).to_html()))

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
884863,36241316,6225,6225,6225.0,36 months,16.49%,220.37,D,D3,Painter,2 years,RENT,27000.0,Source Verified,Jan-2015,Fully Paid,n,debt_consolidation,330xx,FL,18.58,0,Feb-2011,680,684,1,0.0,0.0,3,0,1756,97.6%,4,f,0.00,0.00,7050.460000,7050.46,6225.00,825.46,0.0,0.00,0.0000,4858.17,Dec-2015,684,680,0,0.0,1,Individual,0,0,8357,1800,0.0,0.0,0.0,1,2786.0,44.0,97.6,0,0,32.0,45,28,18,0,28.0,0.0,5.0,0.0,0,2,2,2,2,2,2,2.0,2,3,0,0,0,100.0,100.0,0,0,14307,8357,1800,12507,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,Cash,N,0,0,0,0.0,0.0,0.0
884865,36231724,12000,12000,12000.0,36 months,9.49%,384.34,B,B2,Bartender,4 years,RENT,40000.0,Source Verified,Jan-2015,Fully Paid,n,debt_consolidation,891xx,NV,25.65,1,Oct-2006,665,669,0,8.0,0.0,8,0,8731,73.4%,15,f,0.00,0.00,13217.470001,13217.47,12000.00,1217.47,0.0,0.00,0.0000,7501.84,Jun-2017,674,670,0,8.0,1,Individual,0,0,20671,11900,0.0,0.0,0.0,3,2584.0,2090.0,80.1,1,0,89.0,98,11,11,0,11.0,0.0,11.0,40.0,1,4,5,5,8,5,6,10.0,5,8,0,1,1,80.0,40.0,0,0,28183,20671,10500,16283,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,Cash,N,0,0,0,0.0,0.0,0.0
884866,36421485,4000,4000,4000.0,36 months,8.67%,126.59,B,B1,Lead Custodian,10+ years,MORTGAGE,50000.0,Verified,Jan-2015,Fully Paid,n,car,956xx,CA,12.63,0,Sep-2002,670,674,0,0.0,84.0,11,1,1700,5.6%,30,f,0.00,0.00,4158.020000,4158.02,4000.00,158.02,0.0,0.00,0.0000,3655.51,Oct-2016,674,670,0,0.0,1,Individual,0,0,18979,30100,0.0,0.0,0.0,8,1725.0,12401.0,3.9,0,0,126.0,146,7,7,1,19.0,0.0,3.0,0.0,0,2,5,4,8,14,10,15.0,5,11,0,0,2,100.0,0.0,1,0,58334,18979,12900,28234,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,0,0,0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,Cash,N,0,0,0,0.0,0.0,0.0
884867,35094480,7500,7500,7500.0,36 months,9.49%,240.22,B,B2,Owne

In [5]:
data['loan_status'].unique()

array(['Current', 'Fully Paid', 'In Grace Period', 'Late (31-120 days)',
       'Late (16-30 days)', 'Charged Off', 'Default'], dtype=object)